In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive/hw2

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/hw2


In [2]:
!ls

datasets	   environment.yml	__pycache__	  tf_idf.py
Dov2Vec2.ipynb	   hw2_description.pdf	read_ap.py	  trec.py
download_ap.ipynb  processed_docs.pkl	requirements.txt  WINDOWS.md


In [7]:
import os
import gensim
import csv
from sklearn.model_selection import train_test_split
from tqdm import tqdm
# import trec
import pytrec_eval

import read_ap
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pickle

def main():

    qrels, queries = read_ap.read_qrels()
    docs_by_id = read_ap.get_processed_docs()

    traincorpus = [TaggedDocument(j, [i]) for i, j in docs_by_id.items()]

    model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=1)
    model.build_vocab(traincorpus)
    model.train(traincorpus, total_examples=model.corpus_count, epochs=model.epochs)
    vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
    print(vector)
    
    
    # ranks = []
    # second_ranks = []
    # for doc_id in range(len(traincorpus)):
    #     inferred_vector = model.infer_vector(traincorpus[doc_id].words)
    #     sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    #     rank = [docid for docid, sim in sims].index(doc_id)
    #     ranks.append(rank)

    # second_ranks.append(sims[1])
    
    
    overall_ser = {}

    print("Running Doc2Vec Benchmark")
    # collect results
    for qid in tqdm(qrels): 
        query_text = queries[qid]
        # print(query_text)
        inferred_vector = model.infer_vector(query_text)
        sims = model.docvecs.most_similar([inferred_vector], topn = len(model.docvecs))
        overall_ser[qid] = dict(sims)
    print(list(overall_ser.keys())[0])
    
    return qrels, overall_ser

if __name__ == "__main__":
    qrels, overall_ser = main()
    # evaluator = trec.TrecAPI.evaluate(qrels, overall_ser)
    evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'map', 'ndcg'})
    metrics = trec.evaluate(overall_ser)
    print(evaluator)
    print(metrics)

    
#     Use pickle dump and pickle load to save the models
# pickle.dump( model, open( "modelnumber.p", "wb" ) )
# favorite_color = pickle.load( open( "modelnumber.p", "rb" ) )




Docs already processed. Loading from disk


  0%|          | 0/149 [00:00<?, ?it/s]

[ 0.02925239 -0.01346497  0.00014769  0.0151014   0.01627432 -0.00724757
  0.03680607 -0.00421266  0.00865446  0.00813891 -0.01359113  0.00392202
 -0.01825569  0.00512153 -0.00254896  0.00461481  0.00297795  0.03096714
  0.02303807 -0.00565885 -0.00104     0.00593258 -0.00123206  0.02492198
 -0.0059235  -0.02759317  0.01516818 -0.02080778  0.02166442 -0.01570935
  0.02127239  0.00565895  0.00399433  0.01797415 -0.02457937  0.0212144
 -0.00735703  0.02606635  0.00726126  0.01641699 -0.00130046 -0.00028223
 -0.00189466 -0.01597513  0.01559547 -0.01415886 -0.00403365  0.00628054
  0.00137283 -0.02672526]
Running Doc2Vec Benchmark


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 149/149 [01:16<00:00,  1.98it/s]


101


TypeError: ignored